<a href="https://colab.research.google.com/github/upcarreira/Meu-Portfolio/blob/main/ChatBot_Assistente_Financas_Google_GeminiPRO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChatBot Finanças Expert AI

# 1. Instalação das Bibliotecas:

In [ ]:
!pip install -q -U google-generativeai
!pip install pdfplumber
!pip install nltk
!pip install numpy

# Chatbot Assistente de Finanças com Google Gemini Pro a partir de um PDF de Guia de Finanças

O Código lê um PDF enviado para a raiz do sistema, ele tokeniza os textos e inseri no prompt do contexto da conversa do Gemini com  usuário. Interessante é que ele resumi textos complexos como esse Guia e traduz para o usuário poder conversar com o Chatbot. O grande desafio é a qualidade desse resumo e o maximo numero de Tokens que voce pode inserir no prompt, fizemos esse controle em max_user_input_length=256, pode-se testar diferente valores para testar o modelo.

Sugestões e Melhorias, faça o Fork desse projeto e pode enviar melhorias para mim em meus contatos!

Claudio M. Antonio - Dev

In [ ]:
import google.generativeai as genai
import pdfplumber
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from scipy.spatial.distance import cosine
import numpy as np
from google.colab import userdata
# Carregar a chave da API Gemini (necessário para GenAI)
api_key = userdata.get('GEMINI_KEY')
genai.configure(api_key=api_key)  # Mantenha se quiser usar o GenAI

def preprocess_pdf(pdf_path):
    """
    Função para pré-processar o PDF.

    Args:
        pdf_path: Caminho para o arquivo PDF.

    Returns:
        str: Texto extraído do PDF.
    """
    with open(pdf_path, 'rb') as pdf_file:
        pdf = pdfplumber.open(pdf_file)
        pdf_text = ''
        for page in pdf.pages:
            pdf_text += page.extract_text()
    return pdf_text

def summarize_text(text, ratio=0.2):
    """
    Função para resumir o texto usando a biblioteca NLTK.

    Args:
        text: Texto a ser resumido.
        ratio: Razão de compressão do resumo (entre 0 e 1).

    Returns:
        str: Texto resumido.
    """
    # Download necessário apenas na primeira execução
    nltk.download('stopwords')
    nltk.download('punkt')

    stop_words = stopwords.words('portuguese')
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    sentences = sent_tokenize(text)

    sentence_vectors = []
    for sentence in sentences:
        sentence_words = word_tokenize(sentence.lower())
        sentence_vec = []
        for word in filtered_text:
            if word in sentence_words:
                sentence_vec.append(1)
            else:
                sentence_vec.append(0)
        sentence_vectors.append(sentence_vec)

    sentence_scores = np.apply_along_axis(lambda vec: cosine(vec, np.sum(sentence_vectors, axis=0)), axis=1, arr=np.array(sentence_vectors))
    top_sentences = np.argsort(sentence_scores)[:int(len(sentences) * ratio)]
    summary = ' '.join([sentences[idx] for idx in top_sentences])
    return summary

def chatbot_with_gemini(pdf_path, max_user_input_length=256):
    """
    Função principal para interagir com o chatbot Gemini.

    Args:
        pdf_path: Caminho para o arquivo PDF.
    """
    # Pré-processar o PDF
    preprocessed_text = preprocess_pdf(pdf_path)

    # Resumir o texto do PDF
    summary_text = summarize_text(preprocessed_text)

    # Iniciar a sessão de chat com o modelo Gemini
    chat = genai.GenerativeModel('gemini-pro').start_chat(history=[])

    # Enviar o prompt com o resumo do PDF como o texto inicial
    response = chat.send_message(summary_text)



    # Exibir o prompt inicial do chatbot
    print(f"Chatbot: {response.text}")
    # Perguntar ao usuário o que deseja saber sobre Finanças
    print("Chatbot: O que você deseja saber sobre Finanças? Vamos utilizar o PDF guia de finanças como guia.")


    # Loop principal da conversa
    # Loop principal da conversa
    while True:
      # Obter a entrada do usuário
      user_input = input("Usuário: ")

      # Truncate user input if it exceeds the limit
      if len(user_input) > max_user_input_length:
        user_input = user_input[:max_user_input_length]

      # Processar a entrada do usuário e gerar uma resposta
      response = chat.send_message(user_input)

      # Exibir a resposta do chatbot
      print(f"Chatbot: {response.text}")

# Exemplo de uso
pdf_path = 'guia-financas.pdf'  # Substituir pelo caminho do seu PDF
chatbot_with_gemini(pdf_path)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Chatbot: **Perspectivas Adicionais de Especialistas**

**Warren Buffett**

* Ênfase em investir em empresas sólidas com vantagens competitivas duráveis.
* Adoção de uma abordagem de longo prazo para investimentos.
* Foco em ações subvalorizadas com preços abaixo do valor intrínseco.

**Ray Dalio**

* Importância da diversificação do portfólio.
* Compreensão dos ciclos econômicos para o sucesso financeiro.
* Uso de mega tendências para identificar oportunidades de investimento.

**Adamoradan**

* Importância de uma abordagem estratégica e diversificada para alocação de ativos.
* Ênfase no gerenciamento de riscos e na maximização de retornos.
* Uso de múltiplas ferramentas de avaliação para obter uma visão completa.

**Conceitos Fundamentais de Avaliação Financeira**

**Avaliação**

* Estimar o valor de um ativo, como uma empresa ou ação.

**Rentabilidade**

* Capacidade de uma empresa de gerar lucro em relação a seus ativos, vendas ou patrimônio líquido.

**Cálculo do Valor Presente**

